In [ ]:
#| default_exp sim.mixin

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
class CircuitRunnerMixin:
    """Simulator mixin for running quantum circuits"""
        
    def _apply_gate(self, gate_symbol: str, qubits: (int,tuple)) -> "MeasurementResult":
        """Apply a gate to the `qubits` of the current state."""
        
        gate = getattr(self, gate_symbol)    
        args = (qubits,) if type(qubits)==int else qubits
        return gate(*args)
    
    def run(self, circuit, fault_circuit=None):
        """Apply gates in `circuit` sequentially to current state.
        If `fault_circuit` is specified apply fault gates at end of each tick.
        
        Measurements are stored and executed at the end of a tick.
        Measurement results are saved in the order in which they appear in the
        circuit into an output string."""
        
        msmt_res = []
        for tick_index in range(circuit.n_ticks):
            
            msmts = []
            
            for gate, qubits in circuit[tick_index].items():
                for qubit in qubits:
                    if 'measure' in gate:
                        msmts.append( (gate,qubit) ) # store measure gate, do not execute right away.
                        continue
                    res = self._apply_gate(gate, qubit) # execute gates in tick of circuit
                        
            if fault_circuit:
                for f_gate, f_qubits in fault_circuit[tick_index].items():
                    for f_qubit in f_qubits:
                        self._apply_gate(f_gate, f_qubit) # execute gates in tick of fault circuit
                    
            for gate, qubit in msmts: # exec stored measurement at end of tick.
                res = self._apply_gate(gate, qubit) # Execute measuremnt
                msmt_res.append( int(res.value) ) # Append measurement result to list in order of appearance in tick.

        if msmt_res: 
            return ''.join(map(str, msmt_res))
        else: 
            return None # no measurement